In [2]:
from IPython.display import clear_output

!pip install vosk
!pip install pydub
!pip install openai

clear_output()

In [3]:
import os
import json
import wave
import zipfile

from glob import glob
from openai import OpenAI
from pydub import AudioSegment
from IPython.display import Audio
from vosk import Model, KaldiRecognizer

api_key = ''
client = OpenAI(api_key=api_key)
client

In [7]:
def convert_m4a_to_flac(input_file, output_file):
    # Load the m4a file
    audio = AudioSegment.from_file(input_file, format="m4a")

    # Export as FLAC
    audio.export(output_file, format="flac")

def convert_m4a_to_mono_wav(input_file, output_file):
    # Load the M4A file
    audio = AudioSegment.from_file(input_file, format="m4a")

    # Convert to mono
    mono_audio = audio.set_channels(1)

    # Export as WAV
    mono_audio.export(output_file, format="wav", parameters=["-ac", "1"])

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    """
    Convert an MP3 file to WAV format.

    Parameters:
    mp3_file_path (str): The file path of the MP3 file to convert.
    wav_file_path (str): The file path where the WAV file will be saved.
    """
    # Load the MP3 file
    audio = AudioSegment.from_mp3(mp3_file_path)

    # Export the audio file as WAV
    audio.export(wav_file_path, format="wav")

def generate_content(client, prompt, model='gpt-4-0125-preview', show_tokens=False):
    # Generate the chat completion using the OpenAI API.
    chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=900,  # Uncomment and adjust as necessary.
            temperature=0.0,  # Adjust the randomness of the output (0.0-1.0).
            model=model,  # Model to use for completion.
            response_format={"type": "json_object"},  # The format of the response.
        )

    # Dump the model's response to access detailed information.
    D = chat_completion.model_dump()
    usage = D['usage']

    # If requested, print the token usage information.
    if show_tokens:print(usage)

    # Extract and print the generated message content.
    json_output = D['choices'][0]['message']['content']
    # if show_output:print(json_output)

    # Return the generated content.
    return json_output, usage

def unzip_file(zip_path, extract_to):
    """
    Unzip a ZIP file to the specified directory.

    :param zip_path: Path to the ZIP file.
    :param extract_to: Directory to extract the files into.
    """
    # Ensure the extract_to directory exists
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Open the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents into the directory
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to {extract_to}")

def zip_all_files_in_folder(folder_path, output_zipfile):
    with zipfile.ZipFile(output_zipfile, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for files to keep the directory structure
                relative_path = os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..'))
                zipf.write(os.path.join(root, file), arcname=relative_path)

In [ ]:
# !wget https://alphacephei.com/vosk/models/vosk-model-en-us-0.42-gigaspeech.zip

In [ ]:
# # Example usage
# zip_file_path = 'vosk-model-en-us-0.42-gigaspeech.zip'
# extract_directory = 'models'

# unzip_file(zip_file_path, extract_directory)

In [5]:
glob("*.*m*")

['input.m4a', 'input2.m4a', 'Inoneaudio.mp3']

In [8]:
# convert_m4a_to_mono_wav("Inoneaudio.mp3", "input2.wav")
convert_mp3_to_wav("Inoneaudio.mp3", "input3.wav")

In [ ]:
audio_input = 'input2.wav'

In [10]:
Audio("Inoneaudio.mp3")

In [11]:
model = Model('models/vosk-model-en-us-0.42-gigaspeech')

In [12]:
wf = wave.open(audio_input, "rb")

rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)

In [13]:
results = []
# recognize speech using vosk model
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        part_result = json.loads(rec.Result())
        results.append(part_result)

part_result = json.loads(rec.FinalResult())
results.append(part_result)

# forming a final string from the words
text = ''
for r in results:
    text += r['text'] + ' '

print(f"Vosk thinks you said:\n {text}")

Vosk thinks you said:
 i was like riding with my ninety forty sixty bullets in the trunk space 


In [14]:
Audio(audio_input)

In [15]:
results[0]

{'result': [{'conf': 1.0, 'end': 0.63, 'start': 0.54, 'word': 'i'},
  {'conf': 1.0, 'end': 0.9, 'start': 0.63, 'word': 'was'},
  {'conf': 1.0, 'end': 1.23, 'start': 0.9, 'word': 'like'},
  {'conf': 1.0, 'end': 1.83, 'start': 1.23, 'word': 'riding'},
  {'conf': 1.0, 'end': 2.16, 'start': 1.89, 'word': 'with'},
  {'conf': 1.0, 'end': 2.4, 'start': 2.16, 'word': 'my'},
  {'conf': 1.0, 'end': 3.0, 'start': 2.4, 'word': 'ninety'},
  {'conf': 1.0, 'end': 3.63, 'start': 3.06, 'word': 'forty'},
  {'conf': 1.0, 'end': 4.86, 'start': 3.72, 'word': 'sixty'},
  {'conf': 1.0, 'end': 5.49, 'start': 4.95, 'word': 'bullets'},
  {'conf': 1.0, 'end': 5.7, 'start': 5.52, 'word': 'in'},
  {'conf': 1.0, 'end': 5.94, 'start': 5.7, 'word': 'the'},
  {'conf': 1.0, 'end': 6.48, 'start': 6.0, 'word': 'trunk'},
  {'conf': 1.0, 'end': 7.29, 'start': 6.48, 'word': 'space'}],
 'text': 'i was like riding with my ninety forty sixty bullets in the trunk space'}

In [16]:
info = [(i['word'], i['start'], i['end'], i['end'] - i['start']) for i in results[0]['result']]
words = [i[0] for i in info]

In [17]:
prompt = '''If any of this words contains more than one syllable, return a json with that information, for example

for example the words ["doing", "cat", "dog", "going", "you", "information"] should be returned as the following json: {"words":[("doing", 2), ("going", 2), ("information", 4), ...]}

analyze this list:

''' + f'''{words}'''
print(prompt)

If any of this words contains more than one syllable, return a json with that information, for example

for example the words ["doing", "cat", "dog", "going", "you", "information"] should be returned as the following json: {"words":[("doing", 2), ("going", 2), ("information", 4), ...]}

analyze this list:

['i', 'was', 'like', 'riding', 'with', 'my', 'ninety', 'forty', 'sixty', 'bullets', 'in', 'the', 'trunk', 'space']


In [18]:
text_output, usage = generate_content(client=client, prompt=prompt)
more = json.loads(text_output)
print(more)

{'words': [['riding', 2], ['ninety', 3], ['forty', 2], ['sixty', 2], ['bullets', 2], ['space', 1]]}


In [19]:
words2 = [i for i in more['words'] if i[1]>1]
print(words2)

[['riding', 2], ['ninety', 3], ['forty', 2], ['sixty', 2], ['bullets', 2]]


In [22]:
def cut_audio_to_words(input_audio_path, words_info, folder, lista=[], coef=0):

    os.makedirs(folder, exist_ok=True)

    # Load the entire audio file
    audio = AudioSegment.from_file(input_audio_path)

    words0 = []
    silabas = []

    if len(lista)>0:
        for palabra, silaba in lista:
            words0.append(palabra)
            silabas.append(silaba)

    for word_info in words_info['result']:

        # print(word_info)

        if word_info['word'] not in words0:

            # Extract start and end times in milliseconds
            start_time = word_info['start'] * 1000
            end_time = word_info['end'] * 1000
            # Cut the audio segment for the current word
            word_audio = audio[start_time:end_time]
            # Define the output filename for the word
            output_filename = f"{folder}/{word_info['word']}_{int(start_time)}.wav"
            # Export the audio segment
            word_audio.export(output_filename, format="wav")
            print(f"Exported '{word_info['word']}' to {output_filename}")

        else:

            # Extract start and end times in milliseconds
            start_time = word_info['start'] * 1000
            end_time = word_info['end'] * 1000
            totall = end_time - start_time
            idx = words0.index(palabra)
            partes = silabas[idx]
            div = totall/partes

            # coef = 0.1

            start_idx, end_idx = 0, 0

            for i in range(partes):

                # Cut the audio segment for the current word
                if i==0:
                    start_idx = start_time+(div*(i))
                    end_idx = start_time+(div*(i+1+coef))
                else:
                    start_idx = start_time+(div*(i+coef))
                    end_idx = start_time+(div*(i+1+coef))

                word_audio = audio[start_idx:end_idx]

                # Define the output filename for the word
                output_filename = f"{folder}/{word_info['word']}_{int(start_time)}_{i+1}.wav"
                # Export the audio segment
                word_audio.export(output_filename, format="wav")
                print(f"Exported '{word_info['word']}' to {output_filename}")

        print()

In [21]:
# mkdir sample_2

In [34]:
folderr = 'sample_4'
input_audio_path = 'input2.wav'

words_info = results[0]

cut_audio_to_words(input_audio_path, words_info, folder=folderr, lista=words2, coef=0.25)

Exported 'i' to sample_4/i_540.wav

Exported 'was' to sample_4/was_630.wav

Exported 'like' to sample_4/like_900.wav

Exported 'riding' to sample_4/riding_1230_1.wav
Exported 'riding' to sample_4/riding_1230_2.wav

Exported 'with' to sample_4/with_1890.wav

Exported 'my' to sample_4/my_2160.wav

Exported 'ninety' to sample_4/ninety_2400_1.wav
Exported 'ninety' to sample_4/ninety_2400_2.wav

Exported 'forty' to sample_4/forty_3060_1.wav
Exported 'forty' to sample_4/forty_3060_2.wav

Exported 'sixty' to sample_4/sixty_3720_1.wav
Exported 'sixty' to sample_4/sixty_3720_2.wav

Exported 'bullets' to sample_4/bullets_4950_1.wav
Exported 'bullets' to sample_4/bullets_4950_2.wav

Exported 'in' to sample_4/in_5520.wav

Exported 'the' to sample_4/the_5700.wav

Exported 'trunk' to sample_4/trunk_6000.wav

Exported 'space' to sample_4/space_6480.wav



In [35]:
audios = glob(f"{folderr}/*")
# audios.sort()

for i, aud in enumerate(audios):
    print(i, aud)

0 sample_4/i_540.wav
1 sample_4/was_630.wav
2 sample_4/like_900.wav
3 sample_4/riding_1230_1.wav
4 sample_4/riding_1230_2.wav
5 sample_4/with_1890.wav
6 sample_4/my_2160.wav
7 sample_4/ninety_2400_1.wav
8 sample_4/ninety_2400_2.wav
9 sample_4/forty_3060_1.wav
10 sample_4/forty_3060_2.wav
11 sample_4/sixty_3720_1.wav
12 sample_4/sixty_3720_2.wav
13 sample_4/bullets_4950_1.wav
14 sample_4/bullets_4950_2.wav
15 sample_4/in_5520.wav
16 sample_4/the_5700.wav
17 sample_4/trunk_6000.wav
18 sample_4/space_6480.wav


In [45]:
Audio(audios[14])

In [46]:
# folder_path = 'sample_2'
output_zipfile = 'syllables_4.zip'

zip_all_files_in_folder(folderr, output_zipfile)
print(f"All files in '{folderr}' have been zipped into '{output_zipfile}'")

All files in 'sample_4' have been zipped into 'syllables_4.zip'
